In [1]:

import numpy as np
from flwr.common import NDArrays
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from typing import List
from sklearn.preprocessing import LabelEncoder
from lore_sa.dataset import TabularDataset
import pandas as pd


# This information is needed to create a correct scikit-learn model
UNIQUE_LABELS = []
FEATURES = []
NUM_SERVER_ROUNDS = 5
NUM_CLIENTS = 2
MIN_AVAILABLE_CLIENTS = 2
fds = None  # Cache FederatedDataset

def get_model_parameters(model):
    """Obtener los parámetros del modelo de manera segura."""
    params = [
        max(1, int(model.n_estimators)),  # Asegurar que `n_estimators ≥ 1`
        int(model.max_depth) if model.max_depth is not None else -1,  # Convertir `None` en `-1`
        max(2, int(model.min_samples_split)),  # Asegurar que min_samples_split ≥ 2
        max(1, int(model.min_samples_leaf)),  # Asegurar que min_samples_leaf ≥ 1
    ]
    return params




def set_model_params(model, params):
    """Asegurar que los parámetros son válidos antes de asignarlos."""
    n_estimators_value = max(1, int(params[0]))  # Si `n_estimators=0`, convertirlo a `1`
    max_depth_value = int(params[1]) if int(params[1]) > 0 else None  # Convertir `0` en `None`
    min_samples_split_value = max(2, int(params[2]))  # Si `min_samples_split=0`, convertirlo a `2`
    min_samples_leaf_value = max(1, int(params[3]))  # Si `min_samples_leaf=0`, convertirlo a `1`

    model.set_params(
        n_estimators=n_estimators_value,  # Corregido `n_estimators`
        max_depth=max_depth_value,  # Corregido `max_depth`
        min_samples_split=min_samples_split_value,  # Asegurar que min_samples_split ≥ 2
        min_samples_leaf=min_samples_leaf_value,  # Asegurar que min_samples_leaf ≥ 1
    )




def create_rand_forest_and_instantiate_parameters():
    """Crea un RandomForestClassifier con los parámetros iniciales."""
    return RandomForestClassifier(
        class_weight='balanced',
        criterion='entropy',
        n_estimators=100,
        max_depth=40,
        min_samples_split=2,
        min_samples_leaf=1,
    )



def load_data(partition_id: int, num_partitions: int):
    """Carga los datos del dataset, inicializa UNIQUE_LABELS y FEATURES, y divide en train/test."""
    global fds, UNIQUE_LABELS, FEATURES

    if fds is None:
        partitioner = IidPartitioner(num_partitions=num_partitions)
        fds = FederatedDataset(dataset="hitorilabs/iris", partitioners={"train": partitioner})

    dataset = fds.load_partition(partition_id, "train").with_format("pandas")[:]

    # Convertir la columna objetivo a valores numéricos si es categórica
    target_column = dataset.columns[-1]  

    if dataset[target_column].dtype == "object":
        label_encoder = LabelEncoder()
        dataset[target_column] = label_encoder.fit_transform(dataset[target_column])

    else:

        dataset[target_column] = dataset[target_column].map({0: "Setosa", 1: "Versicolor", 2: "Virginica"})  # Revertir a nombres

    dataset.rename(columns={target_column: "target"}, inplace=True)

    # # Guardar nombres de características antes de convertir a numérico
    # categorical_columns = dataset.select_dtypes(include=["object"]).columns.tolist()
    # if categorical_columns:
    #     dataset = pd.get_dummies(dataset, columns=categorical_columns) 

    dataset.rename(columns={target_column: "target"}, inplace=True)

    # Guardar etiquetas únicas y nombres de características
    if not UNIQUE_LABELS:
        UNIQUE_LABELS = dataset["target"].unique().tolist()

    if not FEATURES:
        FEATURES = dataset.drop(columns=["target"]).columns.tolist()

    # Convertir dataset a formato compatible con LORE
    tabular_dataset = TabularDataset(dataset, "target")

    # Dividir en train/test (80% train - 20% test)
    X = dataset[FEATURES]
    y = dataset["target"]
    split_idx = int(0.8 * len(X))
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    # Devolvemos dataset completo para que LORE pueda acceder a los nombres de las columnas
    return X_train, y_train, X_test, y_test, tabular_dataset


# Cargar datos e inicializar variables automáticamente
X_train, y_train, X_test, y_test, dataset = load_data(partition_id=0, num_partitions=NUM_CLIENTS)

print("UNIQUE_LABELS:", UNIQUE_LABELS)
print("FEATURES:", FEATURES)

# Imprimir los datos reales en el `TabularDataset`
print("\n Contenido del TabularDataset:")
print(dataset.df.head())  # Muestra las primeras filas del dataset

# Imprimir el descriptor del dataset (información sobre las variables)
print("\n Descriptor del TabularDataset:")
print(dataset.descriptor)

2025-03-10 13:19:17,916	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
c:\Users\pablo\anaconda3\Lib\site-packages\flwr_datasets\utils.py:109: UserWarning: The currently tested dataset are ['mnist', 'ylecun/mnist', 'cifar10', 'uoft-cs/cifar10', 'fashion_mnist', 'zalando-datasets/fashion_mnist', 'sasha/dog-food', 'zh-plus/tiny-imagenet', 'scikit-learn/adult-census-income', 'cifar100', 'uoft-cs/cifar100', 'svhn', 'ufldl-stanford/svhn', 'sentiment140', 'stanfordnlp/sentiment140', 'speech_commands', 'LIUM/tedlium', 'flwrlabs/femnist', 'flwrlabs/ucf101', 'flwrlabs/ambient-acoustic-context', 'jlh/uci-mushrooms', 'Mike0307/MNIST-M', 'flwrlabs/usps', 'scikit-learn/iris', 'flwrlabs/pacs', 'flwrlabs/cinic10', 'flwrlabs/caltech101', 'flwrlabs/office-home', 'flwrlabs/fed-isic2019']. Given: hitorilabs/iris.
  warnings.warn(
2025-03-10 13:19:22,580 urllib3.connectionpool 

UNIQUE_LABELS: ['Versicolor', 'Virginica', 'Setosa']
FEATURES: ['petal_length', 'petal_width', 'sepal_length', 'sepal_width']

 Contenido del TabularDataset:
   petal_length  petal_width  sepal_length  sepal_width      target
0           4.8          1.8           5.9          3.2  Versicolor
1           3.5          1.0           5.7          2.6  Versicolor
2           5.6          1.4           6.1          2.6   Virginica
3           1.5          0.2           4.6          3.1      Setosa
4           4.9          1.8           6.3          2.7   Virginica

 Descriptor del TabularDataset:
{'numeric': {'petal_length': {'index': 0, 'min': 1.100000023841858, 'max': 6.699999809265137, 'mean': 3.9026663, 'std': 1.7214837074279785, 'median': 4.5, 'q1': 1.7000000476837158, 'q3': 5.099999904632568}, 'petal_width': {'index': 1, 'min': 0.10000000149011612, 'max': 2.5, 'mean': 1.228, 'std': 0.7334774732589722, 'median': 1.399999976158142, 'q1': 0.3500000089406967, 'q3': 1.7999999523162842}, 's

### Definir el cliente federado con Flower

In [15]:
"""sklearnexample: A Flower / sklearn app."""

import warnings
import numpy as np
from flwr.client import ClientApp, NumPyClient
from sklearn.metrics import log_loss, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from flwr.common import Context
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
import seaborn as sns
import time
import sys  # Para forzar la impresión en consola sin demoras
from threading import Lock
import random

# NUEVO: Importar LORE
from lore_sa.bbox import sklearn_classifier_bbox
from lore_sa.lore import TabularRandomGeneratorLore

def plot_confusion_matrix(y_true, y_pred, labels, client_id):
    """Plotea y muestra la matriz de confusión."""
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    print("\n" + "="*100)
    print(f"[CLIENTE {client_id}] 📊 MATRIZ DE CONFUSIÓN")
    print("-"*100)
    print(cm)
    print("-"*100)
    sys.stdout.flush()
    time.sleep(2)


class FlowerClient(NumPyClient):
    def __init__(self, model, X_train, y_train, X_test, y_test, dataset, client_id):
        self.model = model
        self.X_train = X_train.values
        self.y_train = y_train.values
        self.X_test = X_test.values
        self.y_test = y_test.values
        self.dataset = dataset  # Guardamos dataset completo para LORE
        self.unique_labels = np.unique(y_train)  # Asegurar etiquetas correctas
        self.client_id = client_id  # 🔹 Usamos el partition_id como ID fijo


    def fit(self, parameters, config):
        """Entrenar el modelo antes de cada actualización."""
        set_model_params(self.model, parameters)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            self.model.fit(self.X_train, self.y_train)

        return get_model_parameters(self.model), len(self.X_train), {}
    

    def evaluate(self, parameters, config):
        """Evaluar el modelo y aplicar LORE para explicabilidad."""
        set_model_params(self.model, parameters)

        if not hasattr(self.model, "estimators_") or len(self.model.estimators_) == 0:
            self.model.fit(self.X_train, self.y_train)

        y_pred = self.model.predict(self.X_test)
        y_pred_proba = self.model.predict_proba(self.X_test)

        accuracy = accuracy_score(self.y_test, y_pred)
        precision = precision_score(self.y_test, y_pred, average='weighted', zero_division=1)
        recall = recall_score(self.y_test, y_pred, average='weighted')
        f1 = f1_score(self.y_test, y_pred, average='weighted')
        auc = roc_auc_score(self.y_test, y_pred_proba, multi_class='ovr')
        loss = log_loss(self.y_test, y_pred_proba)

        print("\n" + "="*100)
        print(f"[CLIENTE {self.client_id}] 🔍 INICIO DE EVALUACIÓN")
        print("="*100)
        sys.stdout.flush()
        time.sleep(2)

        # **Matriz de Confusión**
        plot_confusion_matrix(self.y_test, y_pred, labels=self.unique_labels, client_id=self.client_id)

        # **Aplicar LORE**
        print(f"\n[CLIENTE {self.client_id}] 🔍 Aplicando LORE para explicabilidad...")
        print("=" * 100)
        sys.stdout.flush()
        time.sleep(2)

        # Convertimos el modelo en "caja negra" para LORE
        bbox = sklearn_classifier_bbox.sklearnBBox(self.model)

        # Seleccionamos una muestra de prueba del dataset para explicar
        num_row = random.randint(0, len(self.dataset.df) - 1)  # Se puede cambiar para seleccionar una muestra específica, en cada ronda se elegirá una muestra aleatoria
        x = self.dataset.df.iloc[num_row, :-1]  # Excluir la variable objetivo

        # Aplicamos LORE para generar la explicación
        tabularLore = TabularRandomGeneratorLore(bbox, self.dataset)
        explanation = tabularLore.explain(x)

        # Obtener la predicción real del modelo
        predicted_class = self.model.predict([x])[0]  # `x` es la muestra que estamos explicando
        
        # Mostramos la explicación
        print(f"\n[CLIENTE {self.client_id}] 📌 EXPLICACIÓN DEL CLIENTE:\n")
        print(format_explanation(explanation, predicted_class, self.model, self.dataset))
        sys.stdout.flush()
        time.sleep(3)  # Pausa para evitar solapamientos

        return float(loss), len(self.X_test), {
            "Accuracy": float(accuracy),
            "Precision": float(precision),
            "Recall": float(recall),
            "F1_Score": float(f1),
            "AUC": float(auc)
        }

def format_explanation(explanation, predicted_class, model, dataset):
    """Formatea la explicación de LORE incluyendo la predicción y las clases en los contrafactuales."""
    result = "\n" + "="*50 + "\n"
    result += "📌 EXPLICACIÓN DEL MODELO\n"
    result += "="*50 + "\n\n"
    
    # Mostrar la predicción realizada
    result += f"🎯 **Predicción realizada:** {predicted_class}\n\n"

    rule = explanation['rule']
    result += "✅ **Condiciones para la predicción:**\n"
    for condition in rule['premises']:
        attr = condition['attr'].replace("-", " ").capitalize()
        val = condition['val']
        op = condition['op']
        op_text = {"<=": "≤", ">=": "≥", "!=": "NO es"}.get(op, op)
        result += f"  - {attr} {op_text} {val}\n"
    
    result += "\n" + "-"*50 + "\n"
    result += "🔄 **Casos contrafactuales donde el modelo predice otra clase:**\n"
    result += "-"*50 + "\n\n"

    for idx, cf in enumerate(explanation['counterfactuals'], start=1):
        # Aplicamos los cambios del caso contrafactual a la muestra original
        x_cf = dataset.df.iloc[10, :-1].copy()
        for condition in cf['premises']:
            attr = condition['attr']
            val = condition['val']
            x_cf[attr] = val  # Aplicamos los cambios

        # Usamos el modelo para predecir la nueva clase tras el cambio
        counterfactual_class = model.predict([x_cf.values])[0] 

        if counterfactual_class == predicted_class:
            result += f"🛑 **CASO {idx}:** Si se cumplen estas condiciones, la predicción NO cambiaría de **{predicted_class}**\n"
        else:
            result += f"🛑 **CASO {idx}:** Si se cumplen estas condiciones, la predicción cambiaría a **{counterfactual_class}**\n"

        for condition in cf['premises']:
            attr = condition['attr'].replace("-", " ").capitalize()
            val = condition['val']
            op = condition['op']
            op_text = {"<=": "≤", ">=": "≥", "!=": "NO es"}.get(op, op)
            result += f"  - {attr} {op_text} {val}\n"
        result += "\n" + "-"*50 + "\n"

    return result

def client_fn(context: Context):
    """Construir un cliente Flower asegurando que los datos estén cargados correctamente."""
    partition_id = context.node_config["partition-id"]
    num_partitions = context.node_config["num-partitions"]

    # Cargar datos correctamente
    X_train, y_train, X_test, y_test, dataset = load_data(partition_id, num_partitions)

    # Crear modelo RandomForest
    model = create_rand_forest_and_instantiate_parameters()

    return FlowerClient(model, X_train, y_train, X_test, y_test, dataset, client_id=partition_id + 1).to_client()

# 🚀 Crear la aplicación cliente de Flower
client_app = ClientApp(client_fn=client_fn)

# Configurar el Servidor de Flower

In [16]:
"""sklearnexample: A Flower / sklearn app."""

from typing import Dict, List, Tuple

from flwr.common import Context, Metrics, Scalar, ndarrays_to_parameters
from flwr.server import ServerApp, ServerAppComponents, ServerConfig
from flwr.server.strategy import FedAvg

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Dict[str, Scalar]:
    """Compute weighted average.

    It is a generic implementation that averages only over floats and ints and drops the
    other data types of the Metrics.
    """
    # num_samples_list can represent the number of samples
    # or the number of batches depending on the client
    num_samples_list = [n_batches for n_batches, _ in metrics]
    num_samples_sum = sum(num_samples_list)
    metrics_lists: Dict[str, List[float]] = {}
    for num_samples, all_metrics_dict in metrics:
        #  Calculate each metric one by one
        for single_metric, value in all_metrics_dict.items():
            if isinstance(value, (float, int)):
                metrics_lists[single_metric] = []
        # Just one iteration needed to initialize the keywords
        break

    for num_samples, all_metrics_dict in metrics:
        # Calculate each metric one by one
        for single_metric, value in all_metrics_dict.items():
            # Add weighted metric
            if isinstance(value, (float, int)):
                metrics_lists[single_metric].append(float(num_samples * value))

    weighted_metrics: Dict[str, Scalar] = {}
    for metric_name, metric_values in metrics_lists.items():
        weighted_metrics[metric_name] = sum(metric_values) / num_samples_sum

    return weighted_metrics


def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behavior."""

    # penalty = context.run_config.get("penalty", "l1")
    model = create_rand_forest_and_instantiate_parameters()
    ndarrays = get_model_parameters(model)
    global_model_init = ndarrays_to_parameters(ndarrays)

    # Define the strategy
    min_available_clients = context.run_config.get("min-available-clients", MIN_AVAILABLE_CLIENTS)
    strategy = FedAvg(
        min_available_clients=min_available_clients,
        fit_metrics_aggregation_fn=weighted_average,
        evaluate_metrics_aggregation_fn=weighted_average,
        initial_parameters=global_model_init,
    )
    
    # Guardamos la versión original de la función de agregación
    original_aggregate_evaluate = strategy.aggregate_evaluate

    def custom_aggregate_evaluate(server_round, results, failures):
        """Forzar espera antes de continuar con la siguiente ronda del servidor."""
        aggregated_metrics = original_aggregate_evaluate(server_round, results, failures)

        # **Sincronización:** Esperar antes de iniciar la siguiente ronda
        print(f"\n⏳ [SERVIDOR] Esperando 10 segundos antes de iniciar la siguiente ronda...\n")
        time.sleep(10)  # Añadimos una pausa antes de la siguiente ronda

        return aggregated_metrics

    # Sustituye la agregación original por la personalizada (corrigiendo la recursión)
    strategy.aggregate_evaluate = custom_aggregate_evaluate

    num_rounds = context.run_config.get("num-server-rounds", NUM_SERVER_ROUNDS)
    config = ServerConfig(num_rounds=num_rounds)

    return ServerAppComponents(strategy=strategy, config=config)


# Create ServerApp
server_app = ServerApp(server_fn=server_fn)

**Pasos que se realizan en el notebook:**

1. El servidor inicializa el modelo y lo envía a cada uno de los clientes.

2. Cada cliente entrena un RandomForest con su respectivo subconjunto de datos o partición que hemos realizado al principio.

3. Los clientes entrenan, y mandan sus hiperparámetros (Nº de árboles, profundidad, etc.) al servidor.

4. El servidor combina los parámetros y actualiza el modelo global.

5. Se mide el rendimiento del modelo sobre cada cliente, obteniendo también sus contrafactuales y se repite el proceso las rondas que deseemos.


## **Fases del proceso en cada ronda**

**1. El servidor inicia una nueva ronda `(ServerApp)`**

- Llama a `server_fn()`, donde se inicializa la estrategia FedAvg con los parámetros globales.

- Llama a `configure_fit()`, que selecciona qué clientes participarán en la ronda.

---

**2. El servidor envía los parámetros a los clientes**

- Llama a `client_fn(context)`, lo que crea un nuevo cliente (`FlowerClient`).

- Dentro del cliente, se ejecuta:

    ```python

    set_model_params(self.model, parameters)  # Recibe los parámetros globales

    ```

- Cada cliente actualiza su modelo local con los parámetros del servidor.

---


**3. Cada cliente entrena su modelo en sus propios datos**

- Se ejecuta fit() en FlowerClient, que llama a:
    ```python

    self.model.fit(self.X_train, self.y_train)

    ```

- Se entrena un nuevo modelo con los datos locales de cada cliente.

---


**4. Los clientes envían los parámetros actualizados al servidor**

- Después de entrenar, cada cliente ejecuta:
    ```Python

    return get_model_parameters(self.model), len(self.X_train), {}

    ```
- Los parámetros del modelo local se devuelven al servidor.

---

**5. El servidor actualiza el modelo global con FedAvg**

- Recibe los parámetros de cada cliente.

- Llama a: 

    ```python

    strategy.aggregate_fit(server_round, results, failures)

    ```

- Se hace un promedio ponderado de los parámetros recibidos.

- Se actualizan los parámetros globales del modelo.

---


**6. Cada cliente evalúa el modelo**

- Se llama a `evaluate()` en `FlowerClient`, donde:

    ```python

    set_model_params(self.model, parameters)  # Recibe el modelo global actualizado

    y_pred = self.model.predict(self.X_test)

    ```

- Cada cliente evalúa el modelo con su conjunto de prueba.

---

**7. Los clientes envían los resultados de la evaluación al servidor**

- Devuelven métricas Accuracy, Precision, Recall, etc.

- Se ejecuta en el servidor:

    ```python

    strategy.aggregate_evaluate(server_round, results, failures)

    ```

- Se obtiene la evaluación global.

---

**8. Se espera antes de iniciar la siguiente ronda**

- Se ejecuta el `custom_aggregate_evaluate` para hacer una pausa de 10 segundos antes de la siguiente ronda.











# Ejecutar la Simulación Federada


In [17]:
from flwr.simulation import run_simulation
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import ray

ray.shutdown()  # Apagar cualquier sesión previa de Ray
ray.init(local_mode=True)  # Desactiva multiprocessing, usa un solo proceso principal

backend_config = {"num_cpus": 1}

run_simulation(
    server_app=server_app,
    client_app=client_app,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)


2025-02-25 13:13:05,386	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
INFO :      Starting Flower ServerApp, config: num_rounds=5, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
:actor_name:ClientAppActor
:actor_name:ClientAppActor
INFO :      configure_fit: strategy sampled 2 clients (out of 2)
:actor_name:ClientAppActor
:actor_name:ClientAppActor
:actor_name:ClientAppActor
:actor_name:ClientAppActor


:actor_name:ClientAppActor
:actor_name:ClientAppActor
:actor_name:ClientAppActor
:actor_name:ClientAppActor
:actor_name:ClientAppActor
:actor_name:ClientAppActor


INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)



[CLIENTE 1] 🔍 INICIO DE EVALUACIÓN

[CLIENTE 2] 🔍 INICIO DE EVALUACIÓN

[CLIENTE 1] 📊 MATRIZ DE CONFUSIÓN
----------------------------------------------------------------------------------------------------
[[4 0 0]
 [0 8 0]
 [0 0 3]]
----------------------------------------------------------------------------------------------------

[CLIENTE 2] 📊 MATRIZ DE CONFUSIÓN
----------------------------------------------------------------------------------------------------
[[6 0 0]
 [0 2 0]
 [0 1 6]]
----------------------------------------------------------------------------------------------------

[CLIENTE 1] 🔍 Aplicando LORE para explicabilidad...

[CLIENTE 2] 🔍 Aplicando LORE para explicabilidad...

[CLIENTE 1] 📌 EXPLICACIÓN DEL CLIENTE:


📌 EXPLICACIÓN DEL MODELO

🎯 **Predicción realizada:** Virginica

✅ **Condiciones para la predicción:**
  - Petal_length ≤ 5.158425331115723
  - Petal_length > 2.6255003213882446
  - Sepal_length ≤ 6.8689398765563965
  - Sepal_width > 2.83399152755737

INFO :      aggregate_evaluate: received 2 results and 0 failures


...

INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)
INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)



[CLIENTE 1] 🔍 INICIO DE EVALUACIÓN

[CLIENTE 2] 🔍 INICIO DE EVALUACIÓN

[CLIENTE 1] 📊 MATRIZ DE CONFUSIÓN
----------------------------------------------------------------------------------------------------
[[4 0 0]
 [0 8 0]
 [0 0 3]]
----------------------------------------------------------------------------------------------------

[CLIENTE 2] 📊 MATRIZ DE CONFUSIÓN
----------------------------------------------------------------------------------------------------
[[6 0 0]
 [0 2 0]
 [0 0 7]]
----------------------------------------------------------------------------------------------------

[CLIENTE 1] 🔍 Aplicando LORE para explicabilidad...

[CLIENTE 2] 🔍 Aplicando LORE para explicabilidad...

[CLIENTE 2] 📌 EXPLICACIÓN DEL CLIENTE:


📌 EXPLICACIÓN DEL MODELO

🎯 **Predicción realizada:** Versicolor

✅ **Condiciones para la predicción:**
  - Petal_length ≤ 3.001419425010681

--------------------------------------------------
🔄 **Casos contrafactuales donde el modelo predice otra cl

INFO :      aggregate_evaluate: received 2 results and 0 failures



⏳ [SERVIDOR] Esperando 10 segundos antes de iniciar la siguiente ronda...



INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 101.57s
INFO :      	History (loss, distributed):
INFO :      		round 1: 1.2014551129705717
INFO :      		round 2: 6.007275564852859
INFO :      		round 3: 1.2014551129705717
INFO :      		round 4: 4.805820451882287
INFO :      		round 5: 2.220446049250313e-16
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'AUC': [(1, 0.9816849816849818),
INFO :      	         (2, 0.9122405372405373),
INFO :      	         (3, 0.9826388888888888),
INFO :      	         (4, 0.9299242424242424),
INFO :      	         (5, 1.0)],
INFO :      	 'Accuracy': [(1, 0.9666666666666667),
INFO :      	              (2, 0.8333333333333334),
INFO :      	              (3, 0.9666666666666667),
INFO :      	              (4, 0.8666666666666667),
INFO :      	              (5, 1.0)],
INFO :      	 'F1_Score': [(1, 0.9687179487179487),
INFO :      	              (2, 0.8398290598290598),
INFO :      	              (3, 0.967936